# 2.5.1 クロスエントロピー

In [2]:
import torch
from torch import nn

p1 = torch.tensor([1.0, 0.0, 0.0])
p2 = torch.tensor([0.7, 0.2, 0.1])
p3 = torch.tensor([0.1, 0.2, 0.7])

cross_entropy = nn.CrossEntropyLoss()
print(cross_entropy(p1, p2))
print(cross_entropy(p1, p3))

tensor(0.8514)
tensor(1.4514)


## マスク

In [5]:
from torch import Tensor


def create_subsequent_mask(batch_tokens: Tensor):
    sequence_len = batch_tokens.size(1)
    mask = torch.triu(
        torch.full((sequence_len, sequence_len), 1),
        diagonal=1,  # 対角線をいくつ消すか
    )
    mask = mask == 1
    mask = mask.unsqueeze(0)
    return mask

In [7]:
tokens = torch.tensor(
    [
        [1, 2, 3, 4, 5],
        [6, 7, 8, 9, 10],
        [11, 12, 13, 14, 15],
    ]
)

mask = create_subsequent_mask(tokens)
mask

tensor([[[False,  True,  True,  True,  True],
         [False, False,  True,  True,  True],
         [False, False, False,  True,  True],
         [False, False, False, False,  True],
         [False, False, False, False, False]]])